# XGBRegression (P,A,D coordinates to LMA Features)

In [9]:
# Current best:
#datasets/pad/Fs_B_S_DANCE_WALK_0.5sec.csv
"""
Using the identified emotions of each LMA feature set rather than the normal ones
"""

'\nUsing the identified emotions of each LMA feature set rather than the normal ones\n'

## Imports

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [11]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

## Load Data

In [ ]:
dataset = pd.read_csv('datasets/Fs_B_O_DANCE_WALK_KIN_0.5sec.csv')
dataset.head()

### Split into Test and Train data

In [ ]:
train_dataset = dataset.sample(frac=0.9, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

### Split Features from Targets

In [ ]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [ ]:
train_dataset_scaled = scaler.transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

In [ ]:
train_emotions_p = model_p.predict(train_dataset_scaled)
train_emotions_a = model_a.predict(train_dataset_scaled)
train_emotions_d = model_d.predict(train_dataset_scaled)

for i in range(len(train_dataset_scaled)):
    train_emotions.iloc[i] = [train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]]

train_emotions.head()

In [ ]:
train_emotions_OG.head()

In [ ]:
test_emotions_p = model_p.predict(test_dataset_scaled)
test_emotions_a = model_a.predict(test_dataset_scaled)
test_emotions_d = model_d.predict(test_dataset_scaled)

for i in range(len(test_dataset_scaled)):
    test_emotions.iloc[i] = [test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]]


test_emotions.head()

In [ ]:
test_emotions_OG.head()

In [ ]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [ ]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [ ]:
train_emotions.head()

In [ ]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

## Define Model

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=1) 

params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=1500, learning_rate=0.05, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.25, reg_lambda=1.25, gamma=0.01,
                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
                    tree_method='gpu_hist'
                )
    )

## Train Model

In [ ]:
for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    models[i].fit(train_emotions, train_y_sets[i])

## Test Model

### Training Score

In [ ]:
for i in range(len(models)):
    score = models[i].score(train_emotions, train_y_sets[i])
    print(train_y_sets[i].columns[0])
    print("Training score: ", score)

    print()

### Test Set MAE & MSE

In [ ]:
for i in range(len(models)):
    pred_y = models[i].predict(test_emotions)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

In [ ]:
generated_features = []

for i in range(len(models)):
    pred_y = models[i].predict(test_emotions)
    generated_features.append(pred_y)
    
    
rows = []
for i in range(len(generated_features[0])):
    row = []
    for j in range(len(models)):
        row.append(generated_features[j][i])
    rows.append(row)
    
print(len(rows))
        
generated = pd.DataFrame(rows, columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated.head()

In [ ]:
generated_scaled = scaler.transform(generated)

gen_emotions_p = model_p.predict(generated_scaled)
gen_emotions_a = model_a.predict(generated_scaled)
gen_emotions_d = model_d.predict(generated_scaled)

rows = []
for i in range(len(generated_scaled)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

In [ ]:
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

### Prediction Examples

In [ ]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [ ]:
index = 4

real_coordinates = np.asarray([test_X.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(models[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

## Save Models

In [ ]:
# save to JSON
model_p.save_model("models/l2p_dance_model.json")
model_a.save_model("models/l2a_dance_model.json")
model_d.save_model("models/l2d_dance_model.json")

## Load Models

In [ ]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("models/l2p_dance_model_v2.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("models/l2a_dance_model_v2.json")

# Hyperparameter Grid Search with XGBoost

In [63]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


import xgboost as xgb

xgb.set_config(verbosity=2)

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import joblib

## Load Data

In [64]:
dataset = pd.read_csv('datasets/Fs_B_O_DANCE_WALK_KIN_0.5sec.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.684605,0.268051,0.236534,0.478955,0.469381,0.455152,0.351238,0.336392,0.286151,0.278741,...,0.531111,0.677884,1.109843,0.766334,1.537713,1.062221,1.355767,-0.5,0.6,0.9
1,0.438911,0.265254,0.229588,0.283025,0.468163,0.452005,0.350459,0.333469,0.286151,0.278750,...,0.014436,0.021416,1.085836,0.779991,1.575538,1.049393,1.367237,-0.5,0.6,0.9
2,0.440199,0.266441,0.223630,0.309053,0.455287,0.439872,0.343657,0.325378,0.286151,0.278739,...,0.072819,0.191442,1.231576,1.117785,0.133194,0.126177,0.355767,-0.5,0.6,0.9
3,0.500120,0.386881,0.393727,0.452718,0.418828,0.440395,0.324313,0.340373,0.286151,0.278729,...,0.464108,0.605356,1.068333,1.038694,1.638293,1.021144,1.332992,-0.5,0.6,0.9
4,0.562528,0.447009,0.596923,0.488571,0.361697,0.522971,0.286162,0.430045,0.286151,0.278747,...,0.139125,0.207208,1.304347,1.077684,1.563165,0.917321,0.803440,-0.5,0.6,0.9


### Split into Test and Train data

In [65]:
train_dataset = dataset.sample(frac=0.9, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 36166
No Test Samples: 4018


### Split Features from Targets

In [66]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [67]:
train_dataset_scaled = scaler.transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

In [68]:
train_emotions_p = model_p.predict(train_dataset_scaled)
train_emotions_a = model_a.predict(train_dataset_scaled)
train_emotions_d = model_d.predict(train_dataset_scaled)

for i in range(len(train_dataset_scaled)):
    train_emotions.iloc[i] = [train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]]

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
25101,-0.399437,0.251348,-0.100903
10803,-0.599208,-0.298902,-0.300571
16970,-0.350736,0.696850,-0.793898
32453,-0.352074,0.701497,-0.800367
28022,0.594213,0.502033,0.202609


In [69]:
train_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
25101,-0.40,0.25,-0.1
10803,-0.60,-0.30,-0.3
16970,-0.35,0.70,-0.8
32453,-0.35,0.70,-0.8
28022,0.60,0.50,0.2


In [70]:
test_emotions_p = model_p.predict(test_dataset_scaled)
test_emotions_a = model_a.predict(test_dataset_scaled)
test_emotions_d = model_d.predict(test_dataset_scaled)

for i in range(len(test_dataset_scaled)):
    test_emotions.iloc[i] = [test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]]


test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
207,-0.348111,0.696477,-0.777389
17602,-0.599573,-0.300424,-0.294430
27384,0.094744,0.382892,0.110887
39246,-0.394397,0.258712,-0.101554
37900,0.591561,0.490659,0.200044


In [71]:
test_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
207,-0.35,0.70,-0.8
17602,-0.60,-0.30,-0.3
27384,0.60,0.50,0.2
39246,-0.40,0.25,-0.1
37900,0.60,0.50,0.2


In [72]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [73]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [74]:
train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
25101,-0.399437,0.251348,-0.100903
10803,-0.599208,-0.298902,-0.300571
16970,-0.350736,0.696850,-0.793898
32453,-0.352074,0.701497,-0.800367
28022,0.594213,0.502033,0.202609


In [75]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## RandomSearchCV

In [76]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1, 0.15],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [0.85, 1, 1.25],
        'alpha': [0.0, 0.1, 0.25]
        }

n_iter = 200

In [77]:
models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=1500, 
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )
    )

In [ ]:
# Pleasure
# run randomized search

random_searches = []

for i in range(5):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    print(i)
    print()

max_hand_distance


In [ ]:
for i in range(5,10):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    print(i)
    print()

In [ ]:
for i in range(10,15):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    print(i)
    print()

In [ ]:
for i in range(15,20):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    print(i)
    print()

In [ ]:
for i in range(20,27):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    print(i)
    print()

### Results

In [ ]:
best_regressors = []
for i in range(len(random_searches)):
    print(train_y_sets[i].columns[0])
    best_regressor = random_searches[i].best_estimator_

    print(best_regressor.get_params())
    best_regressors.append(best_regressor)
    print()

In [ ]:
for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_emotions)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

In [ ]:
generated_features = []

for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_emotions)
    generated_features.append(pred_y)
    
    
rows = []
for i in range(len(generated_features[0])):
    row = []
    for j in range(len(best_regressors)):
        row.append(generated_features[j][i])
    rows.append(row)
    
print(len(rows))
        
generated = pd.DataFrame(rows, columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated.head()

In [ ]:
generated_scaled = scaler.transform(generated)

gen_emotions_p = model_p.predict(generated_scaled)
gen_emotions_a = model_a.predict(generated_scaled)
gen_emotions_d = model_d.predict(generated_scaled)

rows = []
for i in range(len(generated_scaled)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

In [ ]:
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

In [ ]:
# save to JSON
for i in range(len(best_regressors)):
    best_regressors[i].save_model("models/0.5_sec/xgb/" + test_y_sets[i].columns[0] +"_2.json")

### Predictions

In [ ]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [ ]:
index = 10

real_coordinates = np.asarray([test_X.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(best_regressors[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])